In [4]:
!pip list

Package                       Version              Editable project location
----------------------------- -------------------- -------------------------------------------------------------------------------------
aiohttp                       3.8.4
aiosignal                     1.3.1
alabaster                     0.7.12
amid                          0.7.0
anaconda-client               1.11.0
anaconda-navigator            2.3.1
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.11.7
astropy                       5.1
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-

# Решающее дерево для задачи регрессии


Ваше домашнее задание состоит из 2 частей, задания 1-3 включают в себя имплементацию алгоритма Решающего дерева для задачи регрессии, в задании 4 необходимо применить метод решающего дерева к набору данных стоимости квартир.

# 1. Починить имплементацию решающего дерева

Ниже представлена имплементация алгоритма Решающего дерева для задачи регрессии и пример запуска. В качестве критерия остановки выступает максимальная глубина дерева и минимальное количество наблюдений в листе.

Вам необходимо внести несколько изменений:

    1.1  Сейчас min_samples_leaf не используется, таким образом в листах дерева может оказаться произвольное количество наблюдений, вам необходимо это починить.
    1.2 Для удобства отслеживания числа наблюдений в каждом узле добавьте поле "support" (количество наблюдейний в текущем узле) в словарь `self.tree`.

In [ ]:
import numpy as np

class DecisionTreeRegressor:
    def __init__(self, max_depth, min_samples_leaf):
        """
        max_depth, int - максимальная глубина дерева
        min_samples_leaf, int - минимальное количество наблюдений в листе
        """
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.tree = None
        
    def _split_data(self, X, y, feature_index, threshold):
        left_mask = X[:, feature_index] <= threshold
        right_mask = X[:, feature_index] > threshold
        X_left, y_left = X[left_mask], y[left_mask]
        X_right, y_right = X[right_mask], y[right_mask]
        return X_left, y_left, X_right, y_right
    
    def _mse(self, y):
        return np.mean((y - np.mean(y))**2)
    
    def _best_split(self, X, y):
        best_feature_index, best_threshold = None, None
        best_mse = float('inf')
        n_features = X.shape[1]
        for feature_index in range(n_features):
            feature_values = X[:, feature_index]
            thresholds = np.unique(feature_values)
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self._split_data(X, y, feature_index, threshold)
                mse_left = self._mse(y_left)
                mse_right = self._mse(y_right)
                mse_split = (len(y_left) * mse_left + len(y_right) * mse_right) / len(y)
                if mse_split < best_mse:
                    best_mse = mse_split
                    best_feature_index = feature_index
                    best_threshold = threshold
        return best_feature_index, best_threshold
    
    def _build_tree(self, X, y, depth):
        if depth == self.max_depth:
            return np.mean(y)
        feature_index, threshold = self._best_split(X, y)
        if feature_index is None:
            return np.mean(y)
        X_left, y_left, X_right, y_right = self._split_data(X, y, feature_index, threshold)
        tree = {
            'feature_index': feature_index,
            'threshold': threshold
        }
        tree['left'] = self._build_tree(X_left, y_left, depth + 1)
        tree['right'] = self._build_tree(X_right, y_right, depth + 1)
        return tree
    
    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)
    
    def _predict_one(self, tree, x):
        if isinstance(tree, float):
            return tree
        feature_index, threshold = tree['feature_index'], tree['threshold']
        if x[feature_index] <= threshold:
            return self._predict_one(tree['left'], x)
        else:
            return self._predict_one(tree['right'], x)
        
    def predict(self, X):
        predictions = []
        for x in X:
            prediction = self._predict_one(self.tree, x)
            predictions.append(prediction)
        return np.array(predictions)

In [ ]:
dt = DecisionTreeRegressor(3, 50)

In [1]:
from sklearn.datasets import fetch_openml

In [2]:
X, y = fetch_openml(name="house_prices", as_frame=True, return_X_y=True)
X = X.dropna(axis=1)
X = X.loc[:, X.dtypes == 'float64']
X.drop('Id', axis=1, inplace=True)

In [5]:
X.columns

Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

In [ ]:
dt.fit(X.values, y.values)

In [ ]:
dt.predict(X.values[:5])

In [ ]:
dt.tree

# 2. Воспользуйтесь `line_profiler` 

Текущая имплементация Решающего дерева работает очень медленно. Чтобы локализовать что отнимает так много времени воспользуйтесь `line_profiler`.

    2.1 Что занимает больше всего времени на этапе обучения решающего дерева?
    
https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

In [ ]:
pip install line_profiler

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f dt._build_tree dt._build_tree(X.values[:100], y.values[:100], 7)

# 3. Гистограммы

Вместо того чтобы перебирать все уникальные значения признака в методе `self._best_split` мы будем восстанавливать гистограмму распределения признака и перебирать только уникальные значения в бинах, см. пример:

In [ ]:
x = np.random.normal(0, 1, 200)

In [ ]:
len(np.unique(x))

In [ ]:
bins_height, bins_edges = np.histogram(x, bins=10)
bins_centers = (bins_edges[:-1]+bins_edges[1:])/2

In [ ]:
len(bins_centers)

Т.е. вместо "200" уникальных трешхолдов мы будем использовать только "10".

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
bins_centers

In [ ]:
# Ширина бинов одинакова
bins_edges[:-1]-bins_edges[1:] 

In [ ]:
plt.bar(bins_centers, bins_height, width=0.6);

### Выбор числа бинов

Количество бинов гистограммы может сильно повлиять на то как вы приближаете ваше распределение

In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(12, 9))
n_bins = np.round(np.linspace(1, 50, 12)).astype(int)

for i, ax in enumerate(axs.flatten()):
    ax.hist(x, bins=n_bins[i])
    ax.set_title(f'number of bins is {n_bins[i]}')

### Метод Freedman–Diaconis 

https://en.wikipedia.org/wiki/Freedman%E2%80%93Diaconis_rule

Для выбора числа бинов в гистограмме воспользуйте правилом Freedman–Diaconis:

$$\text{Bin width} = 2 \cdot \frac{IQR(x)}{x^{1/3}}$$

Где IQR это inter quartile range, то есть расстояние между 25 и 75 перцентилем распределения.

    3.1 Метод Freedman–Diaconis дает формулу для ширины бина (одинаковая для всех бинов), а чему в таком случае равно число бинов?
    3.2 Имплементируйте функцию для подсчета числа бинов по формуле Freedman–Diaconis

In [ ]:
# iqr
np.quantile(x, 0.75) - np.quantile(x, 0.25)

### В методе `self._best_split` замените перебор всех уникальных трешхолдов на перебор только по уникальным бинам

Количество бинов подбирайте методом Freedman–Diaconis

    3.3 Засеките время обучения вашего регрессора до и после имплементации гистограмм, насколько быстрее стало обучение?
    3.4 Сравните скорость работы вашей имплементации решающего дерева и имплементации sklearn.tree.DecisionTreeRegressor, с аналогичными гиперпараметрами
    3.5 Перечислите другие способы ускорения алгоритма решающего дерева (кроме приближения распределений признаков гистограмми)

# 4. House pricing

воспользуйтесь набором данных из ДЗ2. Обучите алгоритм Решающего дерева (воспользуйтесь имплементацией из sklearn), не забудьте разделить данные на тренировочную и тестовую выборки. 

    4.1. Влияет ли нормировка признаков на алгоритм Решающего дерева? Почему?
    4.2. Переберите различные значения гиперпараметров решающего дерева с использованием функции GridSearchCV, какое наилучшее качество в терминах mean_squared_error вам удалось получить? Насколько это лучше/хуже качества линейных моделей (на том же train-test разбиении).
    4.3. Сравните важность признаков которую предлагает встроенный метод DecisionTreeRegressor().feature_importances_ с важностью признаков полученной для линейных моделей, прокомментируйте.

## Подготовьте репорт о выполненном домашнем задании

Вы можете вставлять в текст таблицы и/или графики на ваше усмотрение, текст репорта не должен превышать 2 страниц 12 кеглем Times New Roman (крайне желательно 1 страница)

Подготовленный репорт и jupyter notebook c вашим решением необходимо отправить на почту kurmukovai@gmail.com, с темой письма [iitp-intro-ds-2023-ha4-Surname], например [iitp-intro-ds-2023-ha4-Kurmukov] до 23:59:59 МСК 26.03.2023.